## Zadanie - Uzycie _function calling_ do obliczeń matematycznych

W tym zadaniu będziemy używać OpenAI Function Calling do przekazania możliwości wykonywania obliczeń matematycznych do modelu językowego. Dzięki temu LLM będzie mógł prawidłowo rozwiązywać problemy matematyczne, wykonując obliczenia za pomocą biblioteki mathparse.

1. **Cel zadania**: Dać modelowi językowemu możliwość wykonywanie precyzyjnych obliczeń matematycznych.

2. **Wymagania techniczne**:
    - Biblioteka `mathparse` do parsowania i ewaluacji wyrażeń matematycznych
    - API OpenAI (GPT-4o lub podobny model wspierający function calling)

3. **Etapy realizacji**:
    - Stworzenie funkcji pomocniczej do obliczania wyrażeń matematycznych
    - Zdefiniowanie narzędzia (tool) dla modelu GPT
    - Implementacja mechanizmu obsługi wywołań funkcji
    - Stworzenie interfejsu komunikacji z modelem

In [ ]:
import json
from mathparse import mathparse

Przykład użycia _mathparse_:

In [ ]:
mathparse.parse("2 + 2")

### Funkcja `calculate()` - Obliczanie wyrażeń matematycznych

Funkcja `calculate()` pozwala na wykonywanie obliczeń matematycznych z wyrażeń zapisanych w formie tekstu. Funkcja wykorzystuje bibliotekę `mathparse` do analizy i ewaluacji wyrażeń.

#### Przykłady użycia:
- Proste obliczenia: "2 + 2", "5 * 3"
- Funkcje matematyczne: "sin(30)", "sqrt(16)"

In [ ]:
def calculate(math_expression):
    # ... Wasz kod

### Definicja narzędzia (tool) dla modelu

Ten etap obejmuje utworzenie definicji narzędzia, które zostanie przekazane do modelu GPT jako możliwa do wywołania funkcja. Definicja zawiera:

1. **Typ narzędzia** - w tym przypadku "function"
2. **Metadane funkcji**:
    - Nazwa funkcji (`calculate`) - będzie używana przez model do wywołania
    - Szczegółowy opis funkcjonalności
    - Specyfikacja parametrów w formacie JSON Schema

Taka definicja pozwala modelowi zrozumieć:
- Kiedy powinien użyć funkcji obliczeniowej
- Jakie dane musi przekazać
- W jakim formacie powinny być przekazane argumenty

In [ ]:
tools = [
    # ... Wasz kod
]

### Przygotowanie funkcji obsługującej wywołania funkcji

Ten etap polega na stworzeniu funkcji, która obsłuży wywołania narzędzi (tool calls) generowane przez model GPT. Funkcja `handle_function_call()` pełni kluczową rolę w całym procesie:

1. **Przyjmowanie żądań modelu** - odbiera nazwę funkcji oraz argumenty do wykonania obliczeń
2. **Delegowanie wykonania** - wywołuje odpowiednią funkcję (w tym przypadku `calculate()`)
3. **Formatowanie odpowiedzi** - przekształca wynik w format zrozumiały dla modelu

W tym miejscu możemy też pojawić się:

4. **Walidacja danych wejściowych** - sprawdzenie poprawności przekazanych parametrów

In [ ]:
# funkcja obsługuje wywołania funkcji na podstawie nazwy funkcji i argumentów.
def handle_function_call(function_name, arguments):
    print(f"<function_call> Function: {function_name}, Arguments: {arguments}")

    # ... Wasz kod


Poniższa funkcja obsłuży wywołania funkcji zlecone przez model

In [ ]:
def execute_tool_calls(response, messages):
    """
    Wykonuje funkcje zlecone przez model i aktualizuje historię wiadomości.

    Parametry:
    response (openai.types.chat.chat_completion.ChatCompletion): Odpowiedź od modelu zawierająca zlecenia funkcji.
    messages (list): Historia wiadomości.

    Zwraca:
    list: Zaktualizowana historia wiadomości.
    """
    # dodajemy odpowiedź, w której mieliśmy prośbę o wywołanie funkcji, do historii wiadomości
    messages.append(response.choices[0].message.model_dump())

    for tool_call in response.choices[0].message.tool_calls:
        # pobieramy nazwę funkcji i argumenty z odpowiedzi od modelu
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        # wywołanie funkcji na podstawie pobranej nazwy funkcji i argumentów
        result = handle_function_call(name, json.dumps(args))

        # dodajemy wynik funkcji do historii wiadomości
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })

    return messages


Połączenie z modelem

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

Funkcja orkiestrująca całość obsługi poleceń uytkownika.

In [ ]:
def process_user_command(user_command):
    """
    Przetwarza polecenie użytkownika z użyciem GPT-4o oraz function calling.

    Parametry:
    user_command (str): Polecenie użytkownika.

    Zwraca:
    tuple: Para (odpowiedź na polecenie użytkownika, aktualizowana historia wiadomości).
    """

    messages = [
        {"role": "system", "content": "Jesteś pomocnym asystentem. Do obliczeń matematycznych używasz funkcji calulate()."},
        {"role": "user", "content": user_command}
    ]
    
    while True:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0.7,
            max_tokens=16000
        )
        
        message = response.choices[0].message
        
        # Jeśli model żąda wykonania funkcji, wykonujemy je i aktualizujemy historię wiadomości
        if getattr(message, "tool_calls", None):
            messages = execute_tool_calls(response, messages)
        else:
            # Gdy nie ma już żądań funkcji, zwracamy ostateczną odpowiedź modelu.
            return message.content, messages


Wywołajmy to:

In [ ]:
user_command = "ile to jest 2+2*2?"
# user_command = "ile to jest 2 do potęgi 3?"
# user_command = "ile wynosi logarytm naturalny z 20,08553692?"
# user_command = "ile wynosi logarytm o podstawie 3 z 81?"

response_content, messages = process_user_command(user_command)
print(response_content)